In [1]:
import pandas as pd      # primary library for managing and manipulating data
import requests          # library for calling API endpoints
import copy

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("MY_API_KEY")


In [3]:
API_KEY = api_key
base_url = "https://datawebws.usitc.gov/dataweb"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [41]:
query = {
    "savedQueryType": "",
    "isOwner": True,
    "unitConversion": "0",
    "manualConversions": [],
    "reportOptions": {
        "tradeType": "Import",
        "classificationSystem": "HTS"
    },
    "searchOptions": {
        "MiscGroup": {
            "districts": {
                "aggregation": "Aggregate District",
                "districtGroups": {},
                "districts": [],
                "districtsExpanded": [
                    {
                        "name": "All Districts",
                        "value": "all"
                    }
                ],
                "districtsSelectType": "all"
            },
            "importPrograms": {
                "aggregation": None,
                "importPrograms": [],
                "programsSelectType": "all"
            },
            "extImportPrograms": {
                "aggregation": "Aggregate CSC",
                "extImportPrograms": [],
                "extImportProgramsExpanded": [],
                "programsSelectType": "all"
            },
            "provisionCodes": {
                "aggregation": "Aggregate RPCODE",
                "provisionCodesSelectType": "all",
                "rateProvisionCodes": [],
                "rateProvisionCodesExpanded": [],
                "rateProvisionGroups": {
                    "systemGroups": []
                }
            }
        },
        "commodities": {
            "aggregation": "Break Out Commodities",
            "codeDisplayFormat": "YES",
            "commodities": [],
            "commoditiesExpanded": [],
            "commoditiesManual": "",
            "commodityGroups": {
                "systemGroups": [],
                "userGroups": []
            },
            "commoditySelectType": "all",
            "granularity": "10",
            "groupGranularity": None,
            "searchGranularity": None,
            "showHTSValidDetails": ""
        },
        "componentSettings": {
            "dataToReport": [
                "CONS_FIR_UNIT_QUANT",
                "CONS_CUSTOMS_VALUE",
                "CONS_CALC_DUTY"
            ],
            "scale": "1",
            "timeframeSelectType": "fullYears",
            "years": [
                "2005",
                "2004",
                "2003",
                "2002",
                "2001",
                "2000",
                "1999",
                "1998",
                "1997",
                "1996",
                "1995"
            ],
            "startDate": None,
            "endDate": None,
            "startMonth": None,
            "endMonth": None,
            "yearsTimeline": "Monthly"
        },
        "countries": {
            "aggregation": "Break Out Countries",
            "countries": [
                "5700"
            ],
            "countriesExpanded": [
                {
                    "name": "China - CN - CHN",
                    "value": "5700"
                }
            ],
            "countriesSelectType": "list",
            "countryGroups": {
                "systemGroups": [],
                "userGroups": []
            }
        }
    },
    "sortingAndDataFormat": {
        "DataSort": {
            "columnOrder": [
                "COUNTRY",
                "HTS10 & DESCRIPTION",
                "YEAR"
            ],
            "fullColumnOrder": [
                {
                    "checked": False,
                    "disabled": False,
                    "hasChildren": False,
                    "name": "Countries",
                    "value": "COUNTRY",
                    "classificationSystem": "",
                    "groupUUID": "",
                    "items": [],
                    "tradeType": ""
                },
                {
                    "checked": False,
                    "disabled": False,
                    "hasChildren": False,
                    "name": "HTS10 & DESCRIPTION",
                    "value": "HTS10 & DESCRIPTION",
                    "classificationSystem": "",
                    "groupUUID": "",
                    "items": [],
                    "tradeType": ""
                },
                {
                    "checked": False,
                    "disabled": False,
                    "hasChildren": False,
                    "name": "Year",
                    "value": "YEAR",
                    "classificationSystem": "",
                    "groupUUID": "",
                    "items": [],
                    "tradeType": ""
                }
            ],
            "sortOrder": [
                {
                    "sortData": "Countries",
                    "orderBy": "asc",
                    "year": ""
                },
                {
                    "sortData": "HTS10 & DESCRIPTION",
                    "orderBy": "asc",
                    "year": ""
                },
                {
                    "sortData": "Year",
                    "orderBy": "asc",
                    "year": ""
                }
            ]
        },
        "reportCustomizations": {
            "exportCombineTables": False,
            "totalRecords": "20000",
            "exportRawData": True
        }
    },
    "deletedCountryUserGroups": [],
    "deletedCommodityUserGroups": [],
    "deletedDistrictUserGroups": []
}

In [42]:
requestData = query

response = requests.get(base_url+"/api/v2/savedQuery/getAllSavedQueries", 
                        headers=headers, verify=False)
response

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datawebws.usitc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [43]:
response = requests.post(base_url+'/api/v2/report2/runReport', 
                         headers=headers, json=requestData, verify=False)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datawebws.usitc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [44]:
print(response.status_code)
print(response.text)  # see raw response


200
{"dto":null}


In [45]:
resp_json = response.json()
tables = resp_json['dto']['tables']

# Map table index to metric name
metric_names = {
    0: "Customs Value",
    1: "First Unit of Quantity",
    2: "Calculated Duties"
}

all_rows = []

for i, table in enumerate(tables):
    metric = metric_names.get(i, f"Metric_{i}")
    # Columns labels
    col_labels = []
    for col_group in table.get('column_groups', []):
        for col in col_group.get('columns', []):
            col_labels.append(col.get('label', ''))

    # Process rows
    for row_group in table.get('row_groups', []):
        for row in row_group.get('rowsNew', []):
            row_entries = row.get('rowEntries', [])
            # Extract values: handle suppressed if present
            row_values = []
            for entry in row_entries:
                val = entry.get('value', None)
                if 'suppressed' in entry:
                    suppressed = entry.get('suppressed', None)
                    row_values.append(val)  # Keep actual value, ignore suppressed
                else:
                    row_values.append(val)
            all_rows.append([metric] + row_values)

# Add Data Type as first column
final_columns = ['Data Type'] + col_labels
df_combined = pd.DataFrame(all_rows, columns=final_columns)

# Rename last column(s) to 'Value' (optional)
# If you want to flatten months into single 'Value' column, we can melt later
df_combined.to_csv("China.csv", index=False)
print(f"Saved combined CSV with {len(df_combined)} rows and {len(df_combined.columns)} columns")


TypeError: 'NoneType' object is not subscriptable

In [36]:
query2 = {
    "savedQueryType": "",
    "isOwner": True,
    "unitConversion": "0",
    "manualConversions": [],
    "reportOptions": {
        "tradeType": "Import",
        "classificationSystem": "HTS"
    },
    "searchOptions": {
        "MiscGroup": {
            "districts": {
                "aggregation": "Aggregate District",
                "districtGroups": {},
                "districts": [],
                "districtsExpanded": [
                    {
                        "name": "All Districts",
                        "value": "all"
                    }
                ],
                "districtsSelectType": "all"
            },
            "importPrograms": {
                "aggregation": None,
                "importPrograms": [],
                "programsSelectType": "all"
            },
            "extImportPrograms": {
                "aggregation": "Aggregate CSC",
                "extImportPrograms": [],
                "extImportProgramsExpanded": [],
                "programsSelectType": "all"
            },
            "provisionCodes": {
                "aggregation": "Aggregate RPCODE",
                "provisionCodesSelectType": "all",
                "rateProvisionCodes": [],
                "rateProvisionCodesExpanded": [],
                "rateProvisionGroups": {
                    "systemGroups": []
                }
            }
        },
        "commodities": {
            "aggregation": "Break Out Commodities",
            "codeDisplayFormat": "YES",
            "commodities": [],
            "commoditiesExpanded": [],
            "commoditiesManual": "",
            "commodityGroups": {
                "systemGroups": [],
                "userGroups": []
            },
            "commoditySelectType": "all",
            "granularity": "10",
            "groupGranularity": None,
            "searchGranularity": None,
            "showHTSValidDetails": ""
        },
        "componentSettings": {
            "dataToReport": [
                "CONS_FIR_UNIT_QUANT",
                "CONS_CUSTOMS_VALUE",
                "CONS_CALC_DUTY"
            ],
            "scale": "1",
            "timeframeSelectType": "fullYears",
            "years": [
                "2005",
                "2004",
                "2003",
                "2002",
                "2001",
                "2000",
                "1999",
                "1998",
                "1997",
                "1996",
                "1995"
            ],
            "startDate": None,
            "endDate": None,
            "startMonth": None,
            "endMonth": None,
            "yearsTimeline": "Monthly"
        },
        "countries": {
            "aggregation": "Break Out Countries",
            "countries": [],
            "countriesExpanded": [
                {
                    "name": "All Countries",
                    "value": "all"
                }
            ],
            "countriesSelectType": "all",
            "countryGroups": {
                "systemGroups": [],
                "userGroups": []
            }
        }
    },
    "sortingAndDataFormat": {
        "DataSort": {
            "columnOrder": [
                "COUNTRY",
                "HTS10 & DESCRIPTION",
                "YEAR"
            ],
            "fullColumnOrder": [
                {
                    "checked": False,
                    "disabled": False,
                    "hasChildren": False,
                    "name": "Countries",
                    "value": "COUNTRY",
                    "classificationSystem": "",
                    "groupUUID": "",
                    "items": [],
                    "tradeType": ""
                },
                {
                    "checked": False,
                    "disabled": False,
                    "hasChildren": False,
                    "name": "HTS10 & DESCRIPTION",
                    "value": "HTS10 & DESCRIPTION",
                    "classificationSystem": "",
                    "groupUUID": "",
                    "items": [],
                    "tradeType": ""
                },
                {
                    "checked": False,
                    "disabled": False,
                    "hasChildren": False,
                    "name": "Year",
                    "value": "YEAR",
                    "classificationSystem": "",
                    "groupUUID": "",
                    "items": [],
                    "tradeType": ""
                }
            ],
            "sortOrder": [
                {
                    "sortData": "Countries",
                    "orderBy": "asc",
                    "year": ""
                },
                {
                    "sortData": "HTS10 & DESCRIPTION",
                    "orderBy": "asc",
                    "year": ""
                },
                {
                    "sortData": "Year",
                    "orderBy": "asc",
                    "year": ""
                }
            ]
        },
        "reportCustomizations": {
            "exportCombineTables": False,
            "totalRecords": "20000",
            "exportRawData": True
        }
    },
    "deletedCountryUserGroups": [],
    "deletedCommodityUserGroups": [],
    "deletedDistrictUserGroups": []
}

In [37]:
requestData = query2

response = requests.get(base_url+"/api/v2/savedQuery/getAllSavedQueries", 
                        headers=headers, verify=False)
response

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datawebws.usitc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [38]:
response = requests.post(base_url+'/api/v2/report2/runReport', 
                         headers=headers, json=requestData, verify=False)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datawebws.usitc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [39]:
print(response.status_code)
print(response.text)  # see raw response

200
{"dto":null}


In [13]:
# # headers = {
# #     "Content-Type": "application/json",
# #     "Accept": "application/json",
# #     "x-xsrf-token": "817b0850-b736-4be3-9241-3738713e5011",
# #     "Cookie": "JSESSIONID=3A4087CB7E7DFAD7EA2F072BA8B468F5; XSRF-TOKEN=817b0850-b736-4be3-9241-3738713e5011"
# # }

# # Original query template
# query_template = copy.deepcopy(query2)  # your full query JSON

# # Metrics to query individually
# metrics = [
#     ("CONS_CUSTOMS_VALUE", "Customs Value"),
#     ("CONS_FIR_UNIT_QUANT", "First Unit of Quantity"),
#     ("CONS_CALC_DUTY", "Calculated Duties")
# ]

# all_rows = []

# for metric_code, metric_name in metrics:
#     print(f"Querying metric: {metric_name}")
#     requestData = copy.deepcopy(query_template)
#     requestData['searchOptions']['componentSettings']['dataToReport'] = [metric_code]

#     # POST request
#     response = requests.post(
#         base_url + "/api/v2/report2/runReport",
#         headers=headers,
#         json=requestData,
#         verify=False
#     )

#     if response.status_code != 200:
#         print(f"Failed: {metric_name}, Status: {response.status_code}")
#         continue

#     try:
#         resp_json = response.json()
#     except ValueError:
#         print(f"Response for {metric_name} is not JSON!")
#         continue

#     dto = resp_json.get('dto')
#     if not dto or 'tables' not in dto:
#         print(f"No table returned for {metric_name}")
#         print(response.text[:500])
#         continue

#     tables = dto.get('tables', [])
#     if not tables:
#         print(f"No table data for {metric_name}")
#         continue

#     table = tables[0]  # Only one table per metric

#     # Extract column labels
#     col_labels = []
#     for col_group in table.get('column_groups', []):
#         for col in col_group.get('columns', []):
#             col_labels.append(col.get('label', ''))

#     # Extract rows
#     for row_group in table.get('row_groups', []):
#         for row in row_group.get('rowsNew', []):
#             row_entries = row.get('rowEntries', [])
#             row_values = [entry.get('value', None) for entry in row_entries]
#             all_rows.append([metric_name] + row_values)

# # Build final DataFrame
# if all_rows:
#     final_columns = ['Data Type'] + col_labels
#     df_combined = pd.DataFrame(all_rows, columns=final_columns)
#     df_combined.to_csv("all_countries.csv", index=False)
#     print(f"Saved combined CSV with {len(df_combined)} rows and {len(df_combined.columns)} columns")
# else:
#     print("No data returned for any metric.")

In [14]:
requestData = copy.deepcopy(query2)
requestData['searchOptions']['componentSettings']['dataToReport'] = ["CONS_CUSTOMS_VALUE"]

response = requests.post(
    base_url + "/api/v2/report2/runReport",
    headers=headers,
    json=requestData,
    verify=False
)

resp_json = response.json()
tables = resp_json.get('dto', {}).get('tables', [])
all_rows = []

if tables:
    table = tables[0]
    col_labels = []
    for col_group in table.get('column_groups', []):
        for col in col_group.get('columns', []):
            col_labels.append(col.get('label', ''))

    for row_group in table.get('row_groups', []):
        for row in row_group.get('rowsNew', []):
            row_values = [entry.get('value', None) for entry in row.get('rowEntries', [])]
            all_rows.append(["Customs Value"] + row_values)

    final_columns = ['Data Type'] + col_labels
    df_customs = pd.DataFrame(all_rows, columns=final_columns)
    df_customs.to_csv("Customs_Value.csv", index=False)
    print(f"Saved Customs_Value.csv with {len(df_customs)} rows and {len(df_customs.columns)} columns")
else:
    print("No table returned for Customs Value")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datawebws.usitc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Saved Customs_Value.csv with 2476 rows and 15 columns


In [15]:
requestData = copy.deepcopy(query2)
requestData['searchOptions']['componentSettings']['dataToReport'] = ["CONS_FIR_UNIT_QUANT"]

response = requests.post(
    base_url + "/api/v2/report2/runReport",
    headers=headers,
    json=requestData,
    verify=False
)

resp_json = response.json()
tables = resp_json.get('dto', {}).get('tables', [])
all_rows = []

if tables:
    table = tables[0]
    col_labels = []
    for col_group in table.get('column_groups', []):
        for col in col_group.get('columns', []):
            col_labels.append(col.get('label', ''))

    for row_group in table.get('row_groups', []):
        for row in row_group.get('rowsNew', []):
            row_values = [entry.get('value', None) for entry in row.get('rowEntries', [])]
            all_rows.append(["First Unit of Quantity"] + row_values)

    final_columns = ['Data Type'] + col_labels
    df_fir_unit = pd.DataFrame(all_rows, columns=final_columns)
    df_fir_unit.to_csv("First_Unit_Quantity.csv", index=False)
    print(f"Saved First_Unit_Quantity.csv with {len(df_fir_unit)} rows and {len(df_fir_unit.columns)} columns")
else:
    print("No table returned for First Unit of Quantity")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datawebws.usitc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
requestData = copy.deepcopy(query2)
requestData['searchOptions']['componentSettings']['dataToReport'] = ["CONS_CALC_DUTY"]

response = requests.post(
    base_url + "/api/v2/report2/runReport",
    headers=headers,
    json=requestData,
    verify=False
)

resp_json = response.json()
tables = resp_json.get('dto', {}).get('tables', [])
all_rows = []

if tables:
    table = tables[0]
    col_labels = []
    for col_group in table.get('column_groups', []):
        for col in col_group.get('columns', []):
            col_labels.append(col.get('label', ''))

    for row_group in table.get('row_groups', []):
        for row in row_group.get('rowsNew', []):
            row_values = [entry.get('value', None) for entry in row.get('rowEntries', [])]
            all_rows.append(["Calculated Duties"] + row_values)

    final_columns = ['Data Type'] + col_labels
    df_duties = pd.DataFrame(all_rows, columns=final_columns)
    df_duties.to_csv("Calculated_Duties.csv", index=False)
    print(f"Saved Calculated_Duties.csv with {len(df_duties)} rows and {len(df_duties.columns)} columns")
else:
    print("No table returned for Calculated Duties")
